In [2]:
import numpy as np
import math
import os.path
import csv

In [3]:
import imageio
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
! ls '/content/drive/My Drive/Colab Notebooks/6/2'

ls: cannot access '/content/drive/My Drive/Colab Notebooks/6/2': No such file or directory


In [6]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [7]:
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[14], mouth[18])
    C = distance.euclidean(mouth[12], mouth[16])
    mar = (A ) / (C)
    return mar

In [8]:
def mouth_over_eye(eye):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

In [9]:
def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

In [10]:
def getFrame(sec):
    start = 180000 #start time of video
    vidcap.set(cv2.CAP_PROP_POS_MSEC, start + sec*1000)
    hasFrames,image = vidcap.read()
    return hasFrames, image

In [11]:
from scipy.spatial import distance
import cv2
data = []
labels = []
res = []
for j in [54]:
  for i in [0,5,10]:
    vidcap = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/5/' + str(j) +'/' + str(i) + '.mp4')
    sec = 0
    frameRate = 2
    success, image  = getFrame(sec)
    count = 0

    while success and count < 50: 
      landmarks = extract_face_landmarks(image)
      if sum(sum(landmarks)) != 0:
          count += 1
          data.append(landmarks)
          labels.append([i])
          res.append([j])
          sec = sec + frameRate
          sec = round(sec, 2)
          success, image = getFrame(sec)
          print(count)
      else:  
          sec = sec + frameRate
          sec = round(sec, 2)
          success, image = getFrame(sec)
          print("not detected")      
  

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4


KeyboardInterrupt: ignored

In [52]:
print(len(labels))

150


In [53]:
import numpy as np
data = np.array(data)
labels = np.array(labels)
res = np.array(res)

In [54]:
features = []
for d in data:
  eye = d[36:68]
  ear = eye_aspect_ratio(eye)
  mar = mouth_aspect_ratio(eye)
  cir = circularity(eye)
  mouth_eye = mouth_over_eye(eye)
  features.append([ear, mar, cir, mouth_eye])


In [55]:
features = np.array(features)
features=np.append(features,labels,axis=1) 
features=np.append(features,res,axis=1)
features.shape

(150, 6)

In [56]:
np.savetxt("/content/drive/MyDrive/Colab Notebooks/5/54/54.csv", features, delimiter = ",")
np.savetxt("/content/drive/MyDrive/Colab Notebooks/5/54/Copy of 54.csv", features, delimiter = ",")


In [32]:
import pandas as pd
# convert numpy array to dataframe
df = pd.DataFrame(features)
print("\nPandas DataFrame: ")
df


Pandas DataFrame: 


,0,1,2,3,4,5
0,0.295585,0.645828,0.449215,2.184918,0.0,1.0
1,0.197852,0.780875,0.355239,3.946756,0.0,1.0
2,0.244438,0.756843,0.393171,3.096259,0.0,1.0
3,0.279412,0.796041,0.406114,2.848987,0.0,1.0
4,0.308824,0.780521,0.436450,2.527400,0.0,1.0
...,...,...,...,...,...,...
145,0.264093,1.129226,0.414125,4.275872,10.0,1.0
146,0.303786,1.061020,0.467034,3.492657,10.0,1.0
147,0.299614,1.040952,0.427403,3.474311,10.0,1.0
148,0.273821,1.059313,0.424377,3.868634,10.0,1.0


In [33]:
# splitting dataframe by row index
p1 = features.iloc[:150,:]
p2 = features.iloc[150:300,:]
p3 = features.iloc[300:450,:]
p4 = features.iloc[450:600,:]
p5 = features.iloc[600:750,:]
p6 = features.iloc[750:900,:]


AttributeError: ignored

In [34]:
f = [p1,p2,p3]
#f = [p4,p5,p6]
for i in f:
  np.savetxt("/content/drive/MyDrive/Colab Notebooks/6/{}.csv", df, delimiter = ",".format(i))

NameError: ignored

In [35]:
# change file name to save 
#np.save(open('/content/drive/MyDrive/Colab Notebooks/Fold5_part2/3/Data_2.npy', 'wb'),data)
#np.save(open('/content/drive/MyDrive/Colab Notebooks/Fold5_part2/3/Fold5_part2_features_2.npy', 'wb'),features)
#np.save(open('/content/drive/MyDrive/Colab Notebooks/Fold5_part2/3/Fold5_part2_labels_2.npy', 'wb'),labels)
#np.savetxt("/content/drive/MyDrive/Colab Notebooks/Fold5_part2/3/Fold5_part2_labels_2.csv", labels, delimiter = ",")

In [36]:
# how to load npy data file to colab
#a = np.load('Data_32_0.npy')
#b = np.load('Fold3_part2_features_32_0.npy')
#c = np.load('Fold3_part2_labels_32_0.npy')

In [37]:
import pandas as pd
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fold5_part2/1/11.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fold5_part2/2/22.csv')
df3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fold5_part2/3/33.csv')


FileNotFoundError: ignored